## <a href="https://cursos.alura.com.br/course/langchain-python-ferramentas-llm-openai/task/156165"><b>Utilizando LCEL para criar um roteiro de viagens</b></a><br/>

In [1]:
from langchain.prompts import ChatPromptTemplate # Uma interação de múltiplos prompts é um chat, por isso, vamos importar um ChatPromptTemplate

In [2]:
%pip install -qr requirements.txt

Note: you may need to restart the kernel to use updated packages.


#### <b>PASSO 1 - IMPORTS e CRIAÇÃO DA LLM</b>

In [3]:
from langchain_openai import ChatOpenAI
from os import getenv
from dotenv import load_dotenv # CARREGA A VARIÁVEL DE AMBIENTE OPENAI_KEY LIDA DO ARQUIVO .env

load_dotenv() # CARREGANDO O ARQUIVO COM A OPENAI_KEY

llm = ChatOpenAI( # INSTANCIANDO A LLM
                    model="gpt-4.1-mini",
                    temperature=0.5,
                    # 1 - OBTENDO A API KEY POR MEIO DA VARIÁVEL DE AMBIENTE OPENAI_KEY. QUE VAI FICAR ARMAZENADA NO ARQUIVO .env.
                    # 2 - AINDA É NECESSÁRIO CARREGAR ESSE ARQUIVO. VER NA PRIMEIRA CÉLULA DO NOTEBOOK
                    api_key=getenv("OPENAI_KEY")                    
                )

#### <b>PASSO 2 - CRIANDO O <i>PROMPT TEMPLATE</i> E ASSOCIANDO UM PARSER A ELE</b></br> 

<b><ol><li>CRIANDO OS MODELOS</li></ol></b>

<ul><ul><li><b>CRIANDO O PARSER E ASSOCIANDO ELE AO MODELO DE CIDADE</b></li></ul></ul>

In [4]:
from pydantic import Field,BaseModel # pydantic -> Biblioteca para validação de dados. Garante que os dados recebidos ou manipulados estejam no formato correto,
                                     # BaseModel -> Os modelos pydantic são classes que herdam BaseModel (https://docs-pydantic-dev.translate.goog/latest/concepts/models/?_x_tr_sl=en&_x_tr_tl=pt&_x_tr_hl=pt&_x_tr_pto=tc)
                                     # Modelos possuem campos como atributos.
                                     
                                     # Field -> Para personalizar os campos do modelo (https://docs-pydantic-dev.translate.goog/latest/concepts/fields/?_x_tr_sl=en&_x_tr_tl=pt&_x_tr_hl=pt&_x_tr_pto=tc)

class Destino(BaseModel): # A nossa classe vai estender a classe BaseModel, que terá dois campos a cidade e o motivo de visitá-la
    cidade: str = Field(description="cidade a visitar") # Descrição do campo. Apenas informativo
    motivo: str = Field(description="motivo pelo qual é interessante visitar") # Descrição do campo. Apenas informativo  

from langchain import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser # EXISTEM DIVERSOS OUTPUT PARSERS (https://python.langchain.com/docs/concepts/output_parsers/)

parseador = JsonOutputParser(pydantic_object=Destino) # DOCUMENTAÇÃO JsonOutputParser (https://python.langchain.com/docs/how_to/output_parser_json)

template_cidade = PromptTemplate(
                                    template="""Sugira uma cidade, dado o meu interesse por {interesse}.
                                    {formatacao_de_saida_da_ia}""", # AQUI TEMOS UMA VARÍAVEL PARCIAL. UTLIZAÇÃO DA TÉCNICA DE SHOTS PARA PROMPTS
                                    input_variables=["interesse"],
                                    # A VARÍAVEL PARCIAL É UM DICIONÁRIO. FUNCIONA COMO O SHOT
                                    partial_variables={"formatacao_de_saida_da_ia":parseador.get_format_instructions()}, # PASSANDO O PARSEADOR PARA A VARIÁVEL FORMATAÇÃO DE SAÍDA.
                                                                                                   
                                ) # INSTANCIANDO PromptTemplate e INICIANDO A PARTIR DE UM TEMPLATE

<ul><ul><li><b>CRIANDO O MODELO PARA RESTAURANTES</b></li></ul></ul>

In [5]:
template_restaurante = ChatPromptTemplate.from_template("Sugira restaurantes populares entre locais na {cidade}") # INSTANCIANDO ChatPromptTemplate e INICIANDO A PARTIR DE UM TEMPLATE

<ul><ul><li><b>CRIANDO O MODELO CULTURAL</b></li></ul></ul>

In [6]:
template_cultural = ChatPromptTemplate.from_template("Sugira atividades e locais culturais em {cidade}") # INSTANCIANDO ChatPromptTemplate e INICIANDO A PARTIR DE UM TEMPLATE

<ul><ul><li><b>USANDO A <a ref="https://python.langchain.com/docs/concepts/lcel/#composition-syntax">LCEL</a></b></li></ul></ul>
<ul><ul><ul>O resultado de um vai jogando no outro</ul></ul></ul>

In [7]:
chain1 = template_cidade |llm |parseador

In [8]:
# FAZENDO UMA CHAMADA PARA O TEMPLATE
resposta = chain1.invoke({"interesse":"praias"}) # AQUI ESTAMOS CHAMANDO A PRIMEIRA CHAIN, PASSANDO O INTERESSE COMO ENTRADA
print(resposta)

{'cidade': 'Florianópolis', 'motivo': 'Florianópolis é famosa por suas belas praias, com opções para todos os gostos, desde praias tranquilas até locais ideais para surf, além de uma natureza exuberante e ótima infraestrutura turística.'}


In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain.globals import set_debug
set_debug(True)

chain2 = template_restaurante | llm |StrOutputParser() # A saída do llm é uma string, por isso, utilizamos o StrOutputParser
chain = chain1 | chain2 # PARA PODER PASSAR A CIDADE PARA O RESTAURANTE, PRECISAMOS CRIAR UMA NOVA CHAIN QUE RECEBA A PRIMEIRA E A SEGUNDA
resposta = chain.invoke({"interesse":"praias"})
print(resposta)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "interesse": "praias"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "interesse": "praias"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Sugira uma cidade, dado o meu interesse por praias.\n                                    The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {\"properties\": {\"foo\": {\"title\": \"Foo\", \"description\": \"a list of strings\", \"type\": \"array\", \"items\": {\"type\": \"string\"}}}, \"required\": [\"foo\"]}\nthe object {\"foo\": [\"bar\", \"baz\"]} is a well-formatted instance of the schema. The object {\"properties\": {\"foo\": [\"bar\", \"baz\"]}} is not well-formatted.\n\nHere is

#### <b>PASSO 4 - INVOCANDO A CADEIA GERAL</b>

In [10]:
chain3 = template_cultural | llm |StrOutputParser() # A saída do llm é uma string, por isso, utilizamos o StrOutputParser

chain = chain1 | chain2 | chain3 # A saída da cadeia 1 é a entrada da cadeia 2, e assim por diante. Para utilizar a variável cidade

resposta = chain.invoke({"interesse":"praias"}) # DIFERENTE DO QUE ACONTECEU COM O PROMPT TEMPLATE, AQUI INVOCAMOS A CADEIA, E COMTÉM UM TEMPLATE, E ELA INVOCA A LLM
print(resposta)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "interesse": "praias"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "interesse": "praias"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Sugira uma cidade, dado o meu interesse por praias.\n                                    The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {\"properties\": {\"foo\": {\"title\": \"Foo\", \"description\": \"a list of strings\", \"type\": \"array\", \"items\": {\"type\": \"string\"}}}, \"required\": [\"foo\"]}\nthe object {\"foo\": [\"bar\", \"baz\"]} is a well-formatted instance of the schema. The object {\"properties\": {\"foo\": [\"bar\", \"baz\"]}} is not well-formatted.\n\nHere is

<b>RESULTADO DOS PROMPTS</b>